

# Análisis Completo de Regresión Lineal

Este notebook muestra paso a paso cómo realizar un análisis de **Regresión Lineal** empleando un conjunto de datos que contiene:
- **valuationdate**: fecha de valoración (opcional para el modelo)
- **revenues**: ingresos
- **ebit**: utilidad antes de intereses e impuestos
- **depreciation**: depreciación
- **capex**: gastos de capital
- **ncwc**: cambio en capital de trabajo neto
- **bv**: (posiblemente) valor en libros
- **y**: variable objetivo que deseamos predecir (por ejemplo, un valor financiero o de valoración)

Iremos desde la **carga** y **exploración** del dataset hasta la **interpretación** de coeficientes y la **evaluación** de métricas importantes.

In [ ]:
!wget https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv

--2025-03-05 18:51:10--  https://github.com/javierherrera1996/IntroMachineLearning/raw/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv [following]
--2025-03-05 18:51:10--  https://raw.githubusercontent.com/javierherrera1996/IntroMachineLearning/refs/heads/main/PrimerCorte/RegresionLineal/enterprise.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82415 (80K) [text/plain]
Saving to: ‘enterprise.csv’

enterprise.csv      100%[==============

## Paso 0: Importar librerías y configuración

In [16]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


## Paso 1: Carga y Exploración de Datos (EDA)

En este primer bloque, cargaremos nuestro dataset y realizaremos un Análisis Exploratorio básico.

In [17]:


import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('enterprise.csv')
print(df.head())


print(df.info())


print(df.describe())



  valuationdate    revenues          ebit  depreciation      capex  \
0    30/06/2011  86470.7000  10387.500000      6429.000   3512.000   
1    31/12/2010  84754.2000  14031.180800      8658.600  17172.500   
2    30/09/2016  15653.4800    221.347452       647.239    749.478   
3    30/09/2018  62001.3297   7551.761957      1808.830   2702.850   
4    30/09/2014   1242.8300    497.141501       232.027   1552.908   

          ncwc         bv             y  
0  16848.90000  47945.100   91760.80000  
1  12095.10000  66804.400  184461.70000  
2   1468.42800  10128.726   23323.68520  
3  11873.25464  27144.930  130202.79240  
4    149.97100   1463.052   11472.12267  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   valuationdate  1061 non-null   object 
 1   revenues       1061 non-null   float64
 2   ebit           1061 non-null   float64
 

In [18]:
data.head()

,valuationdate,revenues,ebit,depreciation,capex,ncwc,bv,y
0,30/06/2011,0.218942,0.224976,0.050426,0.024623,0.182741,0.116972,0.021887
1,31/12/2010,0.214592,0.245281,0.067899,0.104582,0.176338,0.151996,0.044088
2,30/09/2016,0.039457,0.168322,0.005117,0.008453,0.162024,0.046741,0.005497
3,30/09/2018,0.156925,0.209173,0.014220,0.019887,0.176039,0.078343,0.031094
4,30/09/2014,0.002933,0.169859,0.001863,0.013156,0.160249,0.030648,0.002659


### Estadísticas Descriptivas y Distribuciones

### Preguntas de Reflexión (Exploración)
1. ¿Qué tipo de problema estás intentando resolver al usar `y` como variable objetivo?
2. ¿Notas valores atípicos (outliers) en alguna de las variables? ¿Cómo podría afectar a la regresión lineal?
3. ¿Crees que la columna `valuationdate` podría aportar información para predecir `y`, o la excluirías del análisis?

_Responde con tu razonamiento propio aquí._

In [19]:
# 1. ¿Qué tipo de problema estás intentando resolver al usar y como variable objetivo?
# Se intenta predecir el valor de 'y', que representa una variable objetivo financiera o de valoración.
# El tipo de problema es de regresión, ya que la variable objetivo es continua.
# Sin más contexto sobre 'y', es difícil definir el problema con mayor precisión.


# 2. ¿Notas valores atípicos (outliers) en alguna de las variables? ¿Cómo podría afectar a la regresión lineal?
# Para identificar outliers, es necesario un análisis más profundo de las distribuciones de las variables.
# Un gráfico de caja (boxplot) o un histograma para cada variable numérica ayudaría a visualizar la presencia de valores atípicos.
# Los outliers pueden influir significativamente en la regresión lineal al sesgar los coeficientes estimados y aumentar el error cuadrático medio.
# La regresión lineal es sensible a datos atípicos, por lo que es necesario tratarlos (eliminarlos, transformar la variable, o usar métodos robustos).

# 3. ¿Crees que la columna valuationdate podría aportar información para predecir y, o la excluirías del análisis?
# La columna 'valuationdate' podría aportar información si la variable objetivo 'y' tiene alguna dependencia temporal.
# Por ejemplo, si el valor de 'y' se ve afectado por tendencias o fluctuaciones del mercado que varían con el tiempo, la fecha podría ser un predictor importante.
# Sin embargo, en su forma actual (como fecha), no es directamente útil para la regresión lineal.
# Se podría convertir a variables numéricas (por ejemplo, año, trimestre o diferencias entre fechas) para poder usarla en el modelo.
# Dependiendo de la naturaleza de los datos y la variable objetivo, podría ser conveniente incluirla como una variable categórica (año, mes) o excluirla si no se observa una correlación significativa con 'y' y si genera una complejidad adicional al modelo.


## Paso 2: Preparación y Limpieza de Datos

1. **Manejo de valores faltantes**.
2. **Posible eliminación** o **imputación** de outliers.
3. **Transformaciones** (escalado, log, etc.) si son necesarias.

In [40]:


import pandas as pd
import numpy as np

for col in ['revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv', 'y']:
    df[col].fillna(df[col].mean(), inplace=True)

for col in ['revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv', 'y']:
    upper_limit = df[col].mean() + 2 * df[col].std()
    lower_limit = df[col].mean() - 2 * df[col].std()
    df = df[(df[col] >= lower_limit) & (df[col] <= upper_limit)]

print(f"Número de datos después de la limpieza: {len(df)}")




Número de datos después de la limpieza: 228


<ipython-input-40-1597bd28de8d>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [41]:
data.shape

(945, 8)

### Pregunta de Reflexión (Limpieza)
¿Por qué es relevante tratar valores faltantes y/o outliers **antes** de entrenar el modelo de regresión lineal?

_Responde con tu razonamiento aquí._



Tratar los valores faltantes y los outliers antes de entrenar un modelo de regresión lineal es crucial por varias razones:

1. **Sesgo en los coeficientes:** Los valores faltantes, si no se tratan adecuadamente, pueden inducir un sesgo en la estimación de los coeficientes de la regresión.  Las técnicas de imputación (como la media o la mediana) o la eliminación de filas con valores faltantes pueden mitigar este problema, pero es fundamental elegir la técnica correcta según la naturaleza de los datos.  Si no se manejan correctamente los valores faltantes, el modelo puede aprender patrones erróneos.

2. **Aumento del error:** Los outliers, al ser valores atípicos, ejercen una gran influencia en la regresión lineal, ya que minimiza el error cuadrático medio (MSE).  Estos puntos pueden desviar la línea de regresión, aumentando el error del modelo y reduciendo su precisión en la predicción de nuevos datos que no son outliers. Al eliminarlos o transformarlos, se disminuye la influencia de estos puntos atípicos en la estimación de los parámetros y se obtiene un mejor ajuste del modelo a la mayoría de los datos.

3. **Interpretación errónea:** La presencia de valores faltantes o outliers puede llevar a una interpretación errónea de los resultados del modelo.  Los coeficientes estimados podrían no reflejar las relaciones reales entre las variables.

4. **Mejor rendimiento del modelo:** Al limpiar el dataset, se asegura que el modelo esté entrenado con datos más representativos de la población y se reduce el riesgo de sobreajuste. En consecuencia, se espera que el modelo generalice mejor a nuevos datos, conduciendo a predicciones más precisas y robustas.

En resumen, tratar valores faltantes y outliers mejora la calidad de los datos de entrada, lo que se traduce en una mejor estimación de los coeficientes de la regresión, menor error de predicción, y por lo tanto, un modelo más preciso y confiable.


## Paso 4: Selección de Variables e Implementación del Modelo

1. **Seleccionar** cuáles columnas usarás como variables explicativas (features). Puede que uses todas o hagas una selección.
2. **Asignar** `X` (features) y `y` (variable objetivo).
3. **Dividir** en conjunto de entrenamiento (train) y conjunto de prueba (test).

In [37]:

X = df[['revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv']]
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Preguntas de Reflexión (Selección de Variables)
1. ¿Por qué es importante separar los datos en train y test?
2. ¿Qué pasaría si entrenas y evalúas en el **mismo** conjunto de datos?

_Responde con tu razonamiento._


1. **¿Por qué es importante separar los datos en train y test?**

Separar los datos en conjuntos de entrenamiento (train) y prueba (test) es crucial para evaluar objetivamente el rendimiento de un modelo de aprendizaje automático.  El conjunto de entrenamiento se utiliza para entrenar el modelo, ajustando sus parámetros para minimizar el error en estos datos.  Sin embargo, si evaluamos el modelo con los mismos datos de entrenamiento, no podemos determinar con certeza si el modelo ha aprendido realmente patrones generales o si simplemente se ha memorizado los datos de entrenamiento (sobreajuste). El conjunto de prueba, que el modelo *no ha visto* durante el entrenamiento, proporciona una medida imparcial de su capacidad para generalizar a nuevos datos nunca antes observados.  Esta separación permite estimar la capacidad del modelo para predecir datos futuros con precisión.

2. **¿Qué pasaría si entrenas y evalúas en el mismo conjunto de datos?**

Si entrenas y evalúas un modelo en el mismo conjunto de datos, obtendrás una estimación optimista y sesgada de su rendimiento. El modelo se ajustará perfectamente a los datos de entrenamiento, obteniendo un error muy bajo (o incluso cero en algunos casos), lo que podría llevar a la falsa conclusión de que el modelo es muy preciso.  Sin embargo, este modelo probablemente mostrará un rendimiento deficiente cuando se le presenten nuevos datos, ya que ha aprendido los detalles específicos de los datos de entrenamiento, incluyendo ruido y variaciones específicas, en lugar de aprender las relaciones generales entre las variables. Este fenómeno se conoce como sobreajuste (*overfitting*).  Esencialmente, el modelo habría memorizado los datos en lugar de aprender las reglas subyacentes, y por lo tanto no generaliza bien a nuevos datos.


## Paso 5: Entrenar el Modelo de Regresión Lineal
Usaremos la clase `LinearRegression` de scikit-learn.

In [34]:

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [43]:

intercept = model.intercept_
coef = model.coef_[0]

print(f"Intercept (beta_0): {intercept:.2f}")
print(f"Coeficiente (beta_1): {coef:.2f}")

Intercept (beta_0): 6137.00
Coeficiente (beta_1): 0.11


### Interpretación de Coeficientes
Si la lista de features es `["revenues", "ebit", "depreciation", "capex", "ncwc", "bv"]` y tu `model.coef_` es algo como `[a, b, c, d, e, f]`, entonces:

\[ y \approx \beta_0 + a\cdot revenues + b\cdot ebit + c\cdot depreciation + d\cdot capex + e\cdot ncwc + f\cdot bv \]

Cada coeficiente indica cómo cambia `y` en promedio **por cada unidad** que se incrementa la variable (manteniendo las demás constantes).

In [42]:

feature_names = ["revenues", "ebit", "depreciation", "capex", "ncwc", "bv"]
coefficients = model.coef_

for feature, coef in zip(feature_names, coefficients):
    print(f"Coeficiente de {feature}: {coef:.2f}")


Coeficiente de revenues: 0.11
Coeficiente de ebit: 7.44
Coeficiente de depreciation: 0.40
Coeficiente de capex: 0.27
Coeficiente de ncwc: 0.65
Coeficiente de bv: 0.98


## Paso 6: Evaluación y Validación del Modelo
Usaremos las siguientes métricas:
- **MSE (Mean Squared Error)**
- **MAE (Mean Absolute Error)**
- **R² (Coeficiente de Determinación)**

In [44]:

import numpy as np
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred)) #MAE
r2 = r2_score(y_test, y_pred)


print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


Mean Squared Error (MSE): 424710452.45
Mean Absolute Error (MAE): 14543.71
R-squared (R²): 0.87


### Preguntas de Reflexión (Evaluación)
1. ¿Por qué comparamos las métricas en **Entrenamiento** vs **Prueba**?
2. ¿Qué significa si el error en entrenamiento es **muy** distinto (mucho menor o mucho mayor) que en prueba?
3. ¿Cómo interpretar un **R²** cercano a 1, cercano a 0 o incluso negativo?

_Responde con tu razonamiento._


1. **¿Por qué comparamos las métricas en Entrenamiento vs Prueba?**

Comparar las métricas de rendimiento del modelo en los conjuntos de entrenamiento y prueba es crucial para evaluar la capacidad de generalización del modelo.  El rendimiento en el conjunto de entrenamiento nos indica qué tan bien el modelo se ajusta a los datos con los que fue entrenado. Sin embargo, lo que realmente importa es la capacidad del modelo para predecir valores nuevos y nunca antes vistos.  Al comparar las métricas, podemos detectar si el modelo se ha sobreajustado (overfitting) a los datos de entrenamiento.  Si el error en el conjunto de entrenamiento es mucho menor que en el de prueba, esto sugiere sobreajuste: el modelo aprendió los detalles específicos de los datos de entrenamiento, incluyendo ruido, pero no los patrones generales. Por el contrario, si el error en ambos conjuntos es similar y alto, esto sugiere que el modelo está subajustado (underfitting), y no ha aprendido patrones relevantes de los datos.


2. **¿Qué significa si el error en entrenamiento es muy distinto (mucho menor o mucho mayor) que en prueba?**

* **Error de entrenamiento mucho menor que el de prueba:**  Indica *sobreajuste* (overfitting).  El modelo ha aprendido los datos de entrenamiento demasiado bien, incluyendo el ruido o las particularidades específicas de esos datos, y por lo tanto no generaliza bien a datos nuevos.  En esencia, el modelo ha memorizado los datos de entrenamiento en lugar de aprender los patrones generales subyacentes.

* **Error de entrenamiento mucho mayor que el de prueba:** Esto es menos común, pero podría indicar un problema en la división de los datos o algún error en el proceso de evaluación.  También podría sugerir que el conjunto de entrenamiento es inusualmente representativo, no una buena muestra de la población. Es importante revisar el proceso de selección y división de los datos.  Se debe verificar que los datos de prueba se extraigan aleatoriamente.


3. **¿Cómo interpretar un R² cercano a 1, cercano a 0 o incluso negativo?**

* **R² cercano a 1:** Indica que el modelo explica una gran proporción de la variabilidad en la variable objetivo.  Es una excelente medida de ajuste.  Esto significa que la línea de regresión ajustada se acerca mucho a los puntos de datos reales.
* **R² cercano a 0:** Indica que el modelo no explica una gran parte de la variabilidad de la variable objetivo.  La línea de regresión proporciona una predicción tan buena como usar la media de la variable objetivo.
* **R² negativo:** Esto es inusual y sugiere que el modelo es peor que usar la media de la variable objetivo para hacer predicciones.  Indica un mal ajuste.  Las posibles causas son:  sobreajuste, errores en los datos o un modelo inadecuado.   Se debe revisar cuidadosamente el modelo y el conjunto de datos.


### Pregunta de Reflexión (Residuos)
Si observas un patrón claro en la gráfica de residuos (por ejemplo, forma de U), ¿qué podría indicar respecto a las **asunciones** de la regresión lineal?

_Responde con tu razonamiento._


Si observas un patrón claro en forma de U en la gráfica de residuos, esto indica que la relación entre la variable independiente y la variable dependiente no es lineal, violando una de las asunciones fundamentales de la regresión lineal.  La regresión lineal asume una relación lineal entre las variables, lo que significa que los residuos (las diferencias entre los valores observados y los predichos) deberían estar distribuidos aleatoriamente alrededor de cero y sin mostrar patrones discernibles.

Una forma de U en los residuos sugiere que el modelo lineal subestima los valores de y en ambos extremos del rango de la variable independiente y sobreestima los valores intermedios, o viceversa. Esto implica que una función no lineal (por ejemplo, una función cuadrática o cúbica) podría modelar mejor la relación entre las variables.

En resumen, la forma de U en los residuos indica que la asunción de linealidad se ha violado, y se debe considerar la transformación de las variables, incluir variables cuadráticas o cúbicas como predictores, o utilizar un modelo de regresión no lineal para capturar la verdadera relación entre las variables.

## Paso 8: Interpretación y Conclusiones

1. **Revisa** los coeficientes: ¿son consistentes con la lógica de negocio? (p.ej., si `ebit` aumenta, esperas que `y` aumente o disminuya?).
2. **Observa** el nivel de error y R²: ¿el modelo es suficientemente bueno para su uso práctico?
3. **Considera** la posibilidad de:
   - Añadir más variables.
   - Aplicar **regresiones polinómicas** (X², X³) si sospechas no linealidad.
   - Usar **regularización** (Ridge, Lasso) si hay multicolinealidad o se requieren coeficientes más estables.

In [47]:

print("Coefficients Interpretation:")
for feature, coef in zip(feature_names, coefficients):
    print(f"- {feature}: {coef:.2f}")
    if feature == 'ebit':
        if coef > 0:
            print("  (Consistent: Higher ebit implies higher y)")
        else:
            print("  (Inconsistent: Investigate why higher ebit leads to lower y)")

print(f"\nModel Performance:")
print(f"- MSE: {mse:.2f}")
print(f"- MAE: {mae:.2f}")
print(f"- R-squared: {r2:.2f}")

if r2 < 0.5:
    print("\nModel Improvement Suggestions:")
    print("- Consider adding more relevant features.")
    print("- Explore polynomial regression (X^2, X^3) to capture non-linear relationships.")
    print("- Apply regularization (Ridge, Lasso) to handle multicollinearity or stabilize coefficients.")



Coefficients Interpretation:
- revenues: 0.11
- ebit: 7.44
  (Consistent: Higher ebit implies higher y)
- depreciation: 0.40
- capex: 0.27
- ncwc: 0.65
- bv: 0.98

Model Performance:
- MSE: 424710452.45
- MAE: 14543.71
- R-squared: 0.87


### Preguntas de Reflexión Final
1. ¿Crees que la Regresión Lineal Simple/Múltiple **captura adecuadamente** las relaciones en tus datos?
2. ¿Qué otras técnicas (ej. árboles de decisión, bosques aleatorios, etc.) considerarías para comparar resultados?
3. ¿El dataset es suficientemente grande y variado para hacer conclusiones robustas?

_Responde con tu razonamiento._

In [ ]:


# 1. ¿Crees que la Regresión Lineal Simple/Múltiple captura adecuadamente las relaciones en tus datos?

# La regresión lineal simple/múltiple asume una relación lineal entre las variables predictoras y la variable objetivo.
# Si la relación real es no lineal, la regresión lineal podría no capturar adecuadamente el patrón en los datos, resultando en un bajo R-cuadrado y/o patrones en los residuos. La inspección visual de los residuos, como se mencionó previamente, y la comparación del R-cuadrado con otros modelos pueden indicar si la suposición de linealidad es válida o no.  Si hay patrones no lineales evidentes en los residuos o si el R^2 es bajo, considerar otros modelos sería apropiado.


# 2. ¿Qué otras técnicas (ej. árboles de decisión, bosques aleatorios, etc.) considerarías para comparar resultados?

# Para comparar los resultados y evaluar si la regresión lineal es el modelo más apropiado, se podrían considerar las siguientes técnicas:

# * Árboles de decisión: Son buenos para capturar relaciones no lineales y pueden manejar variables categóricas.
# * Bosques aleatorios:  Combinan varios árboles de decisión para reducir la varianza y el sobreajuste. Generalmente ofrece un buen rendimiento.
# * Gradient Boosting Machines (GBM): Similar a los bosques aleatorios, pero construye árboles de manera secuencial.
# * Redes Neuronales: Podrían capturar relaciones altamente no lineales, pero requieren más datos y pueden ser más complejas de interpretar.
# * Support Vector Machines (SVM):  Potentes para problemas de clasificación y regresión,  pueden ser sensibles a la elección de los hiperparámetros.


# La elección dependerá de la naturaleza de los datos y de las características de la relación entre las variables.
# Un análisis comparativo con estas técnicas ayudaría a determinar qué modelo se ajusta mejor a los datos.

# 3. ¿El dataset es suficientemente grande y variado para hacer conclusiones robustas?

# La robustez de las conclusiones depende del tamaño y la variabilidad del conjunto de datos.
# Un conjunto de datos pequeño o no representativo puede llevar a conclusiones sesgadas o no generalizables. Se debe verificar la cantidad de datos después de la limpieza. Si se han eliminado muchos datos debido a la presencia de valores atípicos o faltantes, es posible que la muestra no sea lo suficientemente representativa.
# Un análisis de la variabilidad de cada variable, a través de indicadores estadísticos y visualizaciones, permitirá determinar si las características de los datos están adecuadamente representadas.  Si el conjunto de datos es pequeño o no variado, las conclusiones del modelo pueden no ser generalizables a la población objetivo, y puede ser necesario recolectar más datos o modificar el alcance de las conclusiones.


## Resumen
En este Notebook hemos:
1. **Explorado** los datos (EDA) y detectado posibles outliers.
2. **Limpieza** de datos y transformaciones opcionales.
3. **Seleccionado** variables y dividido en train/test.
4. **Entrenado** un modelo de Regresión Lineal.
5. **Evaluado** con MSE, MAE, R² y analizado residuos.
6. **Reflexionado** sobre los coeficientes y conclusiones.

¡Has completado un **análisis de regresión lineal** de forma integral! Para afianzar, puedes:
- Ajustar parámetros (aunque la regresión lineal simple en sklearn no tiene muchos).
- Comparar con otros modelos.
- Realizar validación cruzada en lugar de una simple división train/test.

¡Éxitos en tu proyecto de análisis!